# Dependencies

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import yfinance as yf

# Initial fetch + setup

In [25]:
portfolio = {'AAPL': 0.25, 'AMZN': 0.25, 'HEAR': 0.25, 'WEN': 0.25}
window_start = '2018-01-01'
window_end = '2023-01-01'
data_type = 'Adj Close'

data = yf.download(list(portfolio.keys()), start=window_start, end=window_end)[data_type]

[*********************100%%**********************]  4 of 4 completed


Process into returns (closing price -> percentage changes)

In [26]:
returns = data.pct_change().dropna()

returns

Ticker,AAPL,AMZN,HEAR,WEN
Date,,,,
2018-01-03,-0.000174,0.012775,-0.029213,0.023284
2018-01-04,0.004645,0.004476,0.030093,-0.011377
2018-01-05,0.011385,0.016163,0.022472,0.016959
2018-01-08,-0.003714,0.014425,0.006593,-0.010125
2018-01-09,-0.000115,0.004676,-0.028384,0.043321
...,...,...,...,...
2022-12-23,-0.002798,0.017425,0.005755,0.006527
2022-12-27,-0.013878,-0.025924,-0.041488,-0.011673
2022-12-28,-0.030685,-0.014692,-0.010448,-0.007874


#### Expected Return of Asset
Calculate expected returns for each stock. This time we will use historical average just to keep things simple. Other options include:
- CAPM, uses risk in calculation
- DMM or GGMN, better for dividend portfolios with predictable growth

In [27]:
expected_returns = returns.mean()

expected_returns

Ticker
AAPL    0.001141
AMZN    0.000529
HEAR    0.002463
WEN     0.000711
dtype: float64

#### Covariance of Assets
Basically, covariance between financial returns is statistical measure that quantifies degree to which financial assets move together.
$$
Cov_{x,y} = \frac{\sum (x_i - \bar{x})(y_i - \bar{y})}{N-1}
$$

EDIT: NVM pandas can do this for me!!!!

In [28]:
covariance = returns.cov()

covariance

Ticker,AAPL,AMZN,HEAR,WEN
Ticker,,,,
AAPL,0.000445,0.000310,0.000320,0.000187
AMZN,0.000310,0.000509,0.000344,0.000152
HEAR,0.000320,0.000344,0.002863,0.000255
WEN,0.000187,0.000152,0.000255,0.000756


# IMPORTANT PORTFOLIO METRICS
- Expected Return of Portfolio
$$
E(R_p) = w_1 \cdot E(R_1) + w_2 \cdot E(R_2)
$$
- Portfolio Variance
$$
\sigma_p^2 = w_1^2 \cdot \sigma_1^2 + w_2^2 \cdot \sigma_2^2 + 2 \cdot w_1 \cdot w_2 \cdot Cov(R_1, R_2)
$$
- Portfolio Standard Deviation
$$
\sigma_p = \sqrt{\sigma_p^2}
$$
- Sharpe Ratio
$$
\text{Sharpe Ratio} = \frac{E(R_p)-R_f}{\sigma_p}
$$

In [29]:
def metrics(weights, expected_returns, covariance, risk_free_rate):
    portfolio_return = np.dot(weights, expected_returns)
    # NOTE FOR VAR: Need to use weights.T to transpose weights
    portfolio_var = np.dot(weights.T, np.dot(covariance, weights))
    portfolio_std = np.sqrt(portfolio_var)
    sharpe = (portfolio_return - risk_free_rate) / portfolio_std
    return portfolio_return, portfolio_std, sharpe

# TEST
# weights = np.array([0.5, 0.5]) # 50/50 portfolio
# risk_free_rate = 0.02 # 2% risk free rate

# metrics(weights, expected_returns, covariance, risk_free_rate)

# Optimization algorithm for optimal weights
Basically, generate a bunch of possible weight combinations. For each weight combination, calculate portfolio metrics and update best weight if the Sharpe Ratio improves.

In [31]:
####### Define this first #######
risk_free_rate = 0.02
num_points = 100
####### Define this first #######

'''
Bruh this shit way too slow
'''
# def iterate_weights(weight_combinations, num_assets):
#     # Generate all possible combinations of weights
#     if num_assets == 1:
#         yield [1.0]
#     else:
#         for weight in weight_combinations:
#             for remaining_weights in iterate_weights(weight_combinations, num_assets - 1):
#                 yield [weight] + [w * (1 - weight) for w in remaining_weights]

# def optimize(expected_returns, cov_matrix, risk_free_rate, num_points=100):
#     num_assets = len(expected_returns)
#     best_sharpe = -np.inf
#     best_weights = None

#     # Generate all combinations of weights that sum to 1 using linspace
#     weight_combinations = np.linspace(0, 1, num_points)
    
#     # Use itertools to iterate over all possible weight combinations
#     for weights in iterate_weights(weight_combinations, num_assets):
#         weights = np.array(weights)
        
#         if np.isclose(np.sum(weights), 1):  # Ensure weights sum to 1
#             # Calculate portfolio statistics
#             portfolio_return, portfolio_std_dev, sharpe_ratio = metrics(weights, expected_returns, cov_matrix, risk_free_rate)

#             # Update best weights if a higher Sharpe Ratio is found
#             if sharpe_ratio > best_sharpe:
#                 best_sharpe = sharpe_ratio
#                 best_weights = weights

#     return best_weights, best_sharpe

# optimize(expected_returns, covariance, risk_free_rate, num_points)

'''
Using scipy?
'''
def negative_sharpe_ratio(weights, expected_returns, cov_matrix, risk_free_rate):
    return -metrics(weights, expected_returns, cov_matrix, risk_free_rate)[2]

def weight_constraint(weights):
    return np.sum(weights) - 1

bounds = [(0, 1) for _ in range(len(expected_returns))]

initial_guess = [1.0 / len(expected_returns) for _ in range(len(expected_returns))]

constraints = ({'type': 'eq', 'fun': weight_constraint})

result = minimize(
    negative_sharpe_ratio,
    initial_guess,
    args=(expected_returns, covariance, risk_free_rate),
    method='SLSQP',
    bounds=bounds,
    constraints=constraints
)

optimal_weights = result.x

def format_weights_as_percentages(weights):
    return ', '.join([f"{asset}: {weight * 100:.2f}%" for asset, weight in zip(returns.columns, weights)])

print(f'Optimal weights: {format_weights_as_percentages(optimal_weights)}')


array([0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 5.55111512e-17])